# Simulación del Truco

Las **cartas** deben registra el numero, palo y valor de la carta. 
El valor debe reflejar la jerarquía de las cartas en el juego del truco.

In [70]:
def calcular_valor(numero, palo):
    jerarquía = {
    (1,"espada"): 14, (1,"basto"): 13, (7,"espada"): 12, (7,"oro"): 11, 
    (3,"espada"): 10, (3,"basto"): 10, (3,"oro"): 10, (3,"copa"): 10,
    (2,"espada"): 9, (2,"basto"): 9, (2,"oro"): 9, (2,"copa"): 9,
    (1,"oro"): 8, (1,"copa"): 8,
    (12,"espada"): 7, (12,"basto"): 7, (12,"oro"): 7, (12,"copa"): 7,
    (11,"espada"): 6, (11,"basto"): 6, (11,"oro"): 6, (11,"copa"): 6,
    (10,"espada"): 5, (10,"basto"): 5, (10,"oro"): 5, (10,"copa"): 5,
    (7,"basto"): 4, (7,"copa"): 4,
    (6,"espada"): 3, (6,"basto"): 3, (6,"oro"): 3, (6,"copa"): 3,
    (5,"espada"): 2, (5,"basto"): 2, (5,"oro"): 2, (5,"copa"): 2,
    (4,"espada"): 1, (4,"basto"): 1, (4,"oro"): 1, (4,"copa"): 1,
    }
    return jerarquía.get((numero, palo), 0)

class Carta:
    def __init__(self, numero, palo):
        self.numero = numero
        self.palo = palo
        self.valor = calcular_valor(numero, palo)

    def __str__(self):
        return f"{self.numero} de {self.palo} vale = {self.valor}"
    
c = Carta(1, "oro")
print(c)

1 de oro vale = 8


**Mazo**: Representan las cartas españolas de 1 al 12, en los palos de oro, copa, espada y basto.

Debe existir una función que permita barajar las cartas.
y otra función que permita repartir las cartas a los jugadores.

In [88]:
import random

class Mazo:
    def __init__(self):
        self.nuevo_mazo() # Para solucionar problema al faltar cartas para mezclar

    def nuevo_mazo(self):
        self.cartas = [Carta(numero, palo) 
                       for numero in range(1, 13) 
                       if numero != 8 and numero != 9
                       for palo in ["espada", "basto", "oro", "copa"]]
        
    def mezclar(self):
        random.shuffle(self.cartas)

    def repartir(self, cantidad):
        if len(self.cartas) < cantidad:
            self.nuevo_mazo()
            self.mezclar()
        return [self.cartas.pop()
                for _ in range(cantidad)]

m = Mazo()
m.mezclar()
mano = m.repartir(3)

for carta in mano:
    print(carta)

1 de espada vale = 14
11 de copa vale = 6
7 de oro vale = 11


**Jugador**: Deben tener un nombre y las cartas que tiene en la mano. Debe acumular los puntos ganados.

Nota: Para simplificar el problema, el jugador elige la carta a jugar de forma aleatoria.

In [96]:
class Jugador:
    def __init__(self, nombre):
        self.nombre = nombre
        self.cartas = []
        self.puntos = 0
        
    def recibir_cartas(self, cartas):
        self.cartas = cartas
    
    def jugar_carta(self):
        mano_actual = random.choice(self.cartas)
        self.cartas.remove(mano_actual)
        self.puntos += mano_actual.valor
        return mano_actual

j = Jugador("Juan")
j.recibir_cartas(m.repartir(3))
mano_actual = j.jugar_carta()
print(f"{j.nombre} jugó {mano_actual}")

Juan jugó 5 de espada vale = 2


**Partida**: Debe tener dos jugadores, un mazo de cartas y un puntaje. Debe permitir jugar una carta por ronda y definir el ganador de la ronda. Debe permitir jugar hasta que se terminen las cartas del mazo.

In [157]:
class Partida:
    def __init__(self, jugador1, jugador2):
        self.jugador1 = jugador1
        self.jugador2 = jugador2
        self.mazo = Mazo()
        self.puntaje = {jugador1.nombre: 0, jugador2.nombre: 0}

    def jugar_ronda(self):
        self.mazo.mezclar()
        self.jugador1.recibir_cartas(self.mazo.repartir(3))
        self.jugador2.recibir_cartas(self.mazo.repartir(3))

        mano_jugadores = {self.jugador1.nombre: [], self.jugador2.nombre: []} # Almacena las cartas de cada jugador
        for jugador in (self.jugador1, self.jugador2):
            mano_jugadores[jugador.nombre] = [jugador.jugar_carta() for _ in range(3)]
        
        # Contar cuantas cartas de un jugador son mayores a las cartas del otro jugador
        gana_jugador1 = sum(carta.valor > max(carta.valor for carta in mano_jugadores[self.jugador2.nombre]) 
                                              for carta in mano_jugadores[self.jugador1.nombre])
        gana_jugador2 = sum(carta.valor > max(carta.valor for carta in mano_jugadores[self.jugador1.nombre]) 
                                              for carta in mano_jugadores[self.jugador2.nombre])
        
        primer_jugador = random.choice([self.jugador1, self.jugador2])

        if gana_jugador1 >= 2:
            self.puntaje[self.jugador1.nombre] += 1
            print(f"{self.jugador1.nombre} gana la ronda")
        elif gana_jugador2 >= 2:
            print(f"{self.jugador2.nombre} gana la ronda")
        else:
            # Debe ganar quien sea mano, decidí que sea random quien tirara para no complicar el código
            self.puntaje[primer_jugador.nombre] += 1
            print(f"Gana {primer_jugador.nombre} por ser mano!") 

    def jugar(self):
        while self.puntaje[self.jugador1.nombre] < 15 and self.puntaje[self.jugador2.nombre] < 15:
            self.jugar_ronda()

        ganador = self.jugador1.nombre if self.puntaje[self.jugador1.nombre] >= 15 else self.jugador2.nombre
        print(f"El ganador de la partida es {ganador}")

p = Partida(Jugador("Juan"), Jugador("Pedro"))
p.jugar()

Gana Juan por ser mano!
Gana Pedro por ser mano!
Gana Juan por ser mano!
Gana Pedro por ser mano!
Gana Pedro por ser mano!
Gana Juan por ser mano!
Pedro gana la ronda
Gana Juan por ser mano!
Gana Juan por ser mano!
Gana Juan por ser mano!
Juan gana la ronda
Gana Juan por ser mano!
Gana Pedro por ser mano!
Gana Juan por ser mano!
Gana Juan por ser mano!
Gana Pedro por ser mano!
Gana Pedro por ser mano!
Juan gana la ronda
Pedro gana la ronda
Gana Juan por ser mano!
Juan gana la ronda
Pedro gana la ronda
Gana Juan por ser mano!
Juan gana la ronda
El ganador de la partida es Juan
